## About the RabbitMQ message broker in the Smart Village Platform

> "I sent you a POST, why didn't you respond?" **- REST Client**

> "I can't respond right now. I'll probably send you a message later." **- Message Broker**

A message broker like RabbitMQ can receive messages from smart devices and forward them on to an IoT Agent to process the message. Smart devices often support sending AMQP or MQTT protocol messages. RabbitMQ supports AMQP, but the full MQTT version 5 spec used by FIWARE components is not yet supported in RabbitMQ, but will be released in RabbitMQ version 3.13, see [the RabbitMQ releases](https://www.rabbitmq.com/versions.html) and [the RabbitMQ post on MQTT 5](https://blog.rabbitmq.com/posts/2023/07/mqtt5/). Red Hat AMQ Broker is another message broker that supports AMQP and MQTT version 5, but Red Hat AMQ Broker is not available to be deployed in NERC OpenShift, so we'll use RabbitMQ. 


## Deploy the RabbitMQ in NERC OpenShift

If you are curious what the EdgeRabbitMQ custom resource definition looks like that you are deploying in NERC OpenShift, run the command below. 

In [ ]:
%%bash
cat ~/smartvillage-operator/kustomize/overlays/nerc-ocp-prod/ansible/edgerabbitmqs/rabbitmq/edgerabbitmq.yaml
echo DONE

Here are some useful things to note about the configuration of RabbitMQ. 

- `storage:` We disable persistent volume storage on RabbitMQ because we are limited to a maximum of 5 persistent volumes in NERC OpenShift, and we need to save those volumes for the services storing more permanent data. 
- The `resources:` definition defines the memory and CPU requests and limits for our RabbitMQ pod. We have to keep this lower than I would recommend for a production deployment because we have limited resources available in NERC OpenShift for the many running Smart Village services. 

For more information about the EdgeRabbitMQ custom resource definition, [see the full EdgeRabbitMQ schema here](https://github.com/smartabyar-smartvillage/smartvillage-operator/blob/main/config/crd/bases/smartvillage.computate.org_edgerabbitmqs.yaml). 

Run the Ansible Playbook below to deploy RabbitMQ to the cloud. 

In [ ]:
%%bash
ansible-playbook ~/smartvillage-operator/apply-edgerabbitmq.yaml \
  -e ansible_operator_meta_namespace=$(oc get project -o jsonpath={.items[0].metadata.name}) \
  -e crd_path=~/smartvillage-operator/kustomize/overlays/nerc-ocp-prod/ansible/edgerabbitmqs/rabbitmq/edgerabbitmq.yaml
echo DONE

### View RabbitMQ pod details
After running the Ansible Playbook, it will take a minute before the RabbitMQ pod is up and running. Run the command below until the RabbitMQ pod health checks are `READY 1/1` and `STATUS Running`. 

In [ ]:
%%bash
oc -n smart-village-faeeb6c get pod -l app.kubernetes.io/name=rabbitmq
oc -n smart-village-faeeb6c wait pod -l app.kubernetes.io/name=rabbitmq --for=condition=Ready --timeout=2m
oc -n smart-village-faeeb6c get pod -l app.kubernetes.io/name=rabbitmq
echo DONE

### View RabbitMQ pod logs
If your RabbitMQ pod does not reach the STATUS Running, you can run the command below to view the pod logs of RabbitMQ and check for other errors that may have occured. 

In [ ]:
%%bash
oc -n smart-village-faeeb6c logs -l app.kubernetes.io/name=rabbitmq
echo DONE

## Next...
I hope that answers your questions about RabbitMQ in the Smart Village Platform. 
- If you have additional questions or issues, please [create an issue for the course here](https://github.com/smartabyar-smartvillage/smartabyar-smartvillage-nerc-course/issues). 
- Otherwise, please continue to the next notebook [05-about-postgres.ipynb](05-about-postgres.ipynb). 